In [2]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --quiet --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/root/dependencies/awscli-*-py3-none-any.whl'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --upgrade boto3

Note: you may need to restart the kernel to use updated packages.


In [4]:
import boto3
import json

client = boto3.client('bedrock-runtime')

In [5]:
accept = '*/*'
contentType = 'application/json'
modelId = "arn:aws:bedrock:us-west-2:488726636572:provisioned-model/4azc9jbe9ucn"

In [6]:
def query_model(prompt):    
    response = client.invoke_model(body=prompt, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    outputText = response_body.get('results')[0].get('outputText')
    
    return {'output': outputText} 

In [7]:
import pandas as pd

eval_set = pd.read_json("eval_titan.jsonl",lines=True)
eval_set.head()

,input,output
0,kprojects.co.za,constructionservices
1,trinitydev.com,constructionservices
2,yoshua.org,personal
3,stylebish.com,personal
4,lorykt.com,artists


In [13]:
# get result from the fine tuned model

import time
import json

# Create an empty list to store the results
results_list_tuned = []

for i, row in eval_set.iterrows():
    prompt = json.dumps({"inputText": row["input"]})
    result = query_model(prompt = prompt)
    results_list_tuned.append(result)
    
# Create a new DataFrame from the results list
results_list_tuned = pd.DataFrame(results_list_tuned)

results_list_tuned

,output
0,realestate
1,"Sorry, this model is only accessible for Engli..."
2,"Sorry, this model is only accessible for Engli..."
3,"Sorry, this model is only accessible for Engli..."
4,"Sorry, this model is only accessible for Engli..."
...,...
1643,"Sorry, this model is only accessible for Engli..."
1644,"Sorry, this model is only accessible for Engli..."
1645,"Sorry, this model is only accessible for Engli..."
1646,personal_pupplier


In [14]:
results_list_tuned.to_json('Titan-tuned-IC-no-instruct.json')

In [10]:
import pandas as pd

eval_set_2 = pd.read_json("eval_titan_v2.jsonl",lines=True)
eval_set_2.head()

,input,output
0,Classify the following domain name into one of...,constructionservices
1,Classify the following domain name into one of...,constructionservices
2,Classify the following domain name into one of...,personal
3,Classify the following domain name into one of...,personal
4,Classify the following domain name into one of...,artists


In [16]:
# get result from the fine tuned model

import time
import json

# Create an empty list to store the results
results_list_tuned_2 = []

for i, row in eval_set_2.iterrows():
    prompt = json.dumps({"inputText": row["input"]})
    result = query_model(prompt = prompt)
    results_list_tuned_2.append(result)
    
# Create a new DataFrame from the results list
results_list_tuned_2 = pd.DataFrame(results_list_tuned_2)

results_list_tuned_2

,output
0,rentalservices
1,lawyers
2,massage
3,f2yshop
4,g5
...,...
1643,personal_music
1644,childs
1645,cycle
1646,social_ball


In [17]:
results_list_tuned_2.to_json('Titan-tuned-IC-with-instruct.json')

In [8]:
# get result from the base model

modelId = "amazon.titan-text-express-v1"

In [11]:
# get result from the base model

import time
import json

# Create an empty list to store the results
results_list = []

for i, row in eval_set_2.iterrows():
    prompt = json.dumps({"inputText": row["input"]})
    result = query_model(prompt = prompt)
    results_list.append(result)
    
# Create a new DataFrame from the results list
results_list = pd.DataFrame(results_list)

results_list

,output
0,\nIndustry: Technology
1,\nIndustry: Company
2,\nIndustry: Non-profit
3,\nIndustry: Fashion_blog
4,\nIndustry: Technology
...,...
1643,Industry: Technology
1644,Industry: Landscape architects
1645,\nIndustry: Business
1646,\nIndustry: Surveying


In [12]:
results_list.to_json('Titan-base-IC-with-instruct.json')

In [14]:
# analyze result

import pandas as pd

results_tuned = pd.read_json("Titan-tuned-IC-with-instruct.json")
results_base = pd.read_json("Titan-base-IC-with-instruct.json")

In [16]:
results_tuned.rename(columns={'output': 'titan_tuned_output'}, inplace=True)
results_base.rename(columns={'output': 'titan_output'}, inplace=True)

df = eval_set

combined_df = pd.concat([df, 
                         results_base, 
                         results_tuned,], axis=1)
combined_df.head()

,input,output,titan_output,titan_tuned_output
0,kprojects.co.za,constructionservices,\nIndustry: Technology,rentalservices
1,trinitydev.com,constructionservices,\nIndustry: Company,lawyers
2,yoshua.org,personal,\nIndustry: Non-profit,massage
3,stylebish.com,personal,\nIndustry: Fashion_blog,f2yshop
4,lorykt.com,artists,\nIndustry: Technology,g5


In [ ]:
# get another round of results for input with instruct from titan model that fine tuned with instruction

modelId = "arn:aws:bedrock:us-west-2:488726636572:provisioned-model/znvzr83h5fef"

In [ ]:
# get result from the base model

import time
import json

# Create an empty list to store the results
results_list_tuned_3 = []

for i, row in eval_set_2.iterrows():
    prompt = json.dumps({"inputText": row["input"]})
    result = query_model(prompt = prompt)
    results_list_tuned_3.append(result)
    
# Create a new DataFrame from the results list
results_list_tuned_3 = pd.DataFrame(results_list_tuned_3)

results_list_tuned_3

In [30]:
results_list_tuned_3.to_json('Titan-tuned-IC-with-instruct-trained-with-instruct.json')

In [26]:
results_list_tuned_3.rename(columns={'output': 'titan_tuned_2_output'}, inplace=True)
combined_df_2 = pd.concat([combined_df, 
                         results_list_tuned_3,], axis=1)
combined_df_2.head()

,input,output,titan_output,titan_tuned_output,titan_tuned_2_output
0,kprojects.co.za,constructionservices,\nIndustry: Technology,rentalservices,kitchenremodeling
1,trinitydev.com,constructionservices,\nIndustry: Company,lawyers,personal_religion
2,yoshua.org,personal,\nIndustry: Non-profit,massage,personal_resume
3,stylebish.com,personal,\nIndustry: Fashion_blog,f2yshop,shoes
4,lorykt.com,artists,\nIndustry: Technology,g5,divorce


In [29]:
accuracy_titan_tuned = (combined_df_2['titan_tuned_2_output'] == combined_df_2['output']).mean()
print(f"Accuracy of the fine tuned model: {accuracy_titan_tuned * 100:.2f}%")

Accuracy of the fine tuned model: 10.57%
